# 04_MAG_gtdbtk

This document explores the MAG GTDB output. Imported data is the original data files from google drive. This document is written in the Python coding language.

## Load packages and data

In [1]:
import pandas as pd
import os
import sys
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import glob
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt

df = pd.read_csv('/Users/melissaherring/Documents/Bigelow- starting Sept 2022/Virus Project/OMZ_MH_Analysis/Data/mag_data/all_mag_gtdb.csv')
df

,user_genome,classification,fastani_reference,fastani_reference_radius,fastani_taxonomy,fastani_ani,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,...,warnings,sample_name,sample_depth,domain,phyla,class,order,family,genus,species
0,bin_101,d__Archaea;p__Thermoplasmatota;c__Poseidoniia;...,GCA_002731905.1,95.0,d__Archaea;p__Thermoplasmatota;c__Poseidoniia;...,98.12,0.737,GCA_002731905.1,95.0,d__Archaea;p__Thermoplasmatota;c__Poseidoniia;...,...,NaN,JV119,400,Archaea,Thermoplasmatota,Poseidoniia,MGIII,CG-Epi1,UBA8886,UBA8886 sp002731905
1,bin_103,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,JV119,400,Archaea,Nanoarchaeota,Nanoarchaeia,Woesearchaeales,JABIAG01,NaN,NaN
2,bin_107,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,JV119,400,Archaea,Nanoarchaeota,Nanoarchaeia,SCGC-AAA011-G17,JACPNG01,NaN,NaN
3,bin_112,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,NaN,NaN,NaN,NaN,NaN,GCA_016185615.1,95.0,d__Archaea;p__Nanoarchaeota;c__Nanoarchaeia;o_...,...,NaN,JV119,400,Archaea,Nanoarchaeota,Nanoarchaeia,SCGC-AAA011-G17,JACPNG01,JACPNE01,NaN
4,bin_120,d__Archaea;p__Undinarchaeota;c__Undinarchaeia;...,NaN,NaN,NaN,NaN,NaN,GCA_002502135.1,95.0,d__Archaea;p__Undinarchaeota;c__Undinarchaeia;...,...,Genome not assigned to closest species as it f...,JV119,400,Archaea,Undinarchaeota,Undinarchaeia,Undinarchaeales,UBA543,UBA543,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626,bin_95,d__Bacteria;p__Pseudomonadota;c__Alphaproteoba...,GCA_002686135.1,95.0,d__Bacteria;p__Pseudomonadota;c__Alphaproteoba...,96.77,0.856,GCA_002686135.1,95.0,d__Bacteria;p__Pseudomonadota;c__Alphaproteoba...,...,NaN,JV154,140,Bacteria,Pseudomonadota,Alphaproteobacteria,UBA11136,UBA11136,UBA11136,UBA11136 sp002686135
627,bin_96,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,NaN,NaN,NaN,NaN,NaN,GCA_022572185.1,95.0,d__Bacteria;p__Pseudomonadota;c__Gammaproteoba...,...,Genome not assigned to closest species as it f...,JV154,140,Bacteria,Pseudomonadota,Gammaproteobacteria,Woeseiales,Woeseiaceae,JACZWL01,NaN
628,bin_97,Unclassified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No bacterial or archaeal marker,JV154,140,Unclassified,NaN,NaN,NaN,NaN,NaN,NaN
629,bin_98,Unclassified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,No bacterial or archaeal marker,JV154,140,Unclassified,NaN,NaN,NaN,NaN,NaN,NaN


## How many MAGs were classified in total?

In [15]:
total = len(df)
total

631

## How many MAGs were classified to at least that classification level?

In [10]:
# function to count how many were classified based on an input dataframe and taxonomic level

def count_classified(df, level):
    
    level_prefix = level[0] + '__' # create level prefixes
    
    subdf = df[(df[level] != 'Unclassified') & (~df[level].isna()) & (df[level] != '') & (df[level] != level_prefix)] # all conditions that level is unannotated
   
    return len(subdf)  # returns number of rows with annotation for level

In [8]:
level_counts = [] # set up empty level counts string\

levels = ['domain','phyla','class','order','family','genus','species'] # create a levels list

# for loop that counts how many were classified using the count_classified function created above
for level in levels:
    lcount = count_classified(df, level)
    
    print('There are', lcount, 'MAGs annotated to', level,".") 
    
    level_counts.append(lcount)

There are 566 MAGs annotated to domain .
There are 452 MAGs annotated to phyla .
There are 452 MAGs annotated to class .
There are 450 MAGs annotated to order .
There are 441 MAGs annotated to family .
There are 350 MAGs annotated to genus .
There are 216 MAGs annotated to species .


In [16]:
taxdict = {'tax_level': levels, 'num_MAGs_classified': level_counts} # create a dictionary with the number classified for each level

GTDB_tax = pd.DataFrame(data = taxdict)

GTDB_tax['percent_classified'] = GTDB_tax['num_MAGs_classified']/total*100 # add a column to the dataframe that is the percent of MAGs

GTDB_tax

,tax_level,num_MAGs_classified,percent_classified
0,domain,566,89.698891
1,phyla,452,71.632330
2,class,452,71.632330
3,order,450,71.315372
4,family,441,69.889065
5,genus,350,55.467512
6,species,216,34.231379


## Plots

In [ ]:
# count plot
GTDB_tax.plot(kind='bar', x='tax_level', y='num_MAGs_classified',legend=None)
plt.xlabel('Taxonomic classification')
plt.ylabel('number of MAGs')
plt.title('Assignment of MAGs at different taxonomic levels with GTDB')
plt.tight_layout()

In [ ]:
# percent plot
GTDB_tax.plot(kind='bar', x='tax_level', y='percent_classified',legend=None)
plt.xlabel('Taxonomic classification')
plt.ylabel('percent of MAGs')
plt.title('Percent of MAGs classified at different taxonomic levels with GTDB')
plt.tight_layout()

## What are the most common classifications?

In [ ]:
Counter(combo['classification']).most_common()[:10]

In [ ]:
# top 5 groups in each level
for taxa in ['domain', 'phyla','class','order','family','genus','species']:
    print(combo.groupby(taxa)['user_genome'].count().sort_values(ascending=False)[:5])
    print("\n")

#### Phyla

In [ ]:
phyla_df = combo.groupby('phyla', as_index=False)['user_genome'].count().sort_values(by = 'user_genome',ascending=False)
top_phyla = phyla_df [:10]

top_phyla.plot('phyla','user_genome',kind = 'bar',legend=None)
plt.xlabel('Phyla')
plt.ylabel('Count')
plt.title('Number of MAGs belonging to the top 10 most abundant phyla with GTDB')

#### Class

In [ ]:
class_df = combo.groupby('class', as_index=False)['user_genome'].count().sort_values(by = 'user_genome',ascending=False)
top_class = class_df [:10]

top_class.plot('class','user_genome',kind = 'bar',legend=None)
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Number of MAGs belonging to the top 10 most abundant classes with GTDB')

#### Order

In [ ]:
order_df = combo.groupby('order', as_index=False)['user_genome'].count().sort_values(by = 'user_genome',ascending=False)
top_order = order_df [:10]

top_order.plot('order','user_genome',kind = 'bar',legend=None)
plt.xlabel('Order')
plt.ylabel('Count')
plt.title('Number of MAGs belonging to the top 10 most abundant orders with GTDB')

## Plot tax level

In [ ]:
top_10_class = list(df.groupby('class', as_index = False)['user_genome'].count().sort_values(by = 'user_genome', ascending = False)[:10]['class'])
top_10_class 

In [ ]:
plot_class = []

for item in df['class']:
    if item in top_10_class:
        plot_class.append(item)
    else:
        plot_class.append('Other')

df['plot_class'] = plot_class

In [ ]:
# alternative: list comprehension solution
df['plot_class'] = [item if item in top_10_class else 'Other' for item in df['plot_class']]

In [ ]:
class_counts = df.groupby(['sample_depth', 'plot_class'], as_index = False)['user_genome'].count().pivot(columns = 'plot_class', index = 'sample_depth', values = 'user_genome')


class_counts.plot.bar(stacked = True)

plt.legend(bbox_to_anchor=(1.1, 1.05))


In [ ]:
class_counts

In [ ]:
column_order = ['Other'] + top_10_class

class_pcts = class_counts.div(class_counts.sum(axis=1), axis=0)
class_pcts = round(class_pcts * 100, 1)



class_pcts[column_order].plot.bar(stacked = True)
plt.legend(bbox_to_anchor=(1.1, 1.05))

In [ ]:
class_pcts